In [650]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import model_selection, svm

import seaborn as sns; sns.set_theme()
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# importando nosso dataframe

In [651]:
#carrega a base de treino
df=pd.read_csv(r"../input/churn-data/BankChurners.csv",encoding='latin-1')

# Removendo colunas desnecessarias

as duas ultimas colunas causam sobreajuste dos dados.

In [652]:
x = df.drop(['Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1',
             'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2'
             ,'Attrition_Flag','CLIENTNUM'],axis='columns')

# Como os dados se comportam ?

In [653]:
df.sample(n=10)

In [654]:
x.info()

## Não apresentarei boxplots que não possuem otliers para não sobrecarregar a quantidade de boxplot no arquivo python.

In [655]:
boxplot = df.boxplot(column=['Months_on_book'])

In [656]:
boxplot = df.boxplot(column=['Months_Inactive_12_mon'])

In [657]:
sns.histplot((df['Total_Revolving_Bal']), kde=True)

In [658]:
boxplot = df.boxplot(column=['Credit_Limit'])

In [659]:
sns.histplot((df['Credit_Limit']), kde=True)

In [660]:

sns.histplot((df['Avg_Open_To_Buy']), kde=True)

In [661]:
boxplot = df.boxplot(column=['Avg_Open_To_Buy'])

In [662]:
sns.histplot((df['Total_Amt_Chng_Q4_Q1']), kde=True)

In [663]:
boxplot = df.boxplot(column=['Total_Trans_Ct'])

In [664]:
sns.histplot((df['Total_Ct_Chng_Q4_Q1']), kde=True)

# Tratando alguns outliers

In [665]:
#esta função encontra os limites superiores e inferiores
# e retorna o dataframe dentro dos limites
def boxplott(df,text):
    #df=df.sort_values(text)
    Q1=df[text].quantile(q=0.25)
    Q3=df[text].quantile(q=0.75)
    FIQ = Q3-Q1

    LF = Q1 - 1.5 * FIQ
    LS = Q3 + 1.5 * FIQ
    
    #print('LS : ', LS)
    #print('LF : ', LF)
    df.loc[df[text] > LS , text] = LS
    df.loc[df[text] < LF , text] = LF
    return df

### Vamos converter algumas colunas para seu logaritmo, isso faz com que reduza levemente os outliers da coluna especifica.
### outra abordagem é quando em uma coluna há poucos outlier aplico a função boxplott que reduz os outliers. Em algumas situações pode se mostrar ineficiente mas aqui aumentou levemente a accuracy.

In [666]:
x['Total_Amt_Chng_Q4_Q1']=np.log(x['Total_Amt_Chng_Q4_Q1']+1)
x['Avg_Open_To_Buy']=np.log(x['Avg_Open_To_Buy']+1)
x['Total_Ct_Chng_Q4_Q1']=np.log(x['Total_Ct_Chng_Q4_Q1']+1)
x = boxplott(x,'Months_on_book')
x = boxplott(x,'Months_Inactive_12_mon')
x = boxplott(x,'Contacts_Count_12_mon')
x = boxplott(x,'Total_Trans_Ct')

# O que utilizar? Label encoder ou OneHotEncoder?

Testei ambas abordagens e cheguei a conclusão que Label Encoder foi mais promissor se tratando dos dois modelos de ML que utilizei. Talvez em uma rede neural a abordagem mudaria.

### Label encoder

In [667]:

labelencoder = LabelEncoder()
x['Gender'] = labelencoder.fit_transform(x['Gender'])
x['Education_Level'] = labelencoder.fit_transform(x['Education_Level'])
x['Marital_Status'] = labelencoder.fit_transform(x['Marital_Status'])
x['Income_Category'] = labelencoder.fit_transform(x['Income_Category'])
x['Card_Category'] = labelencoder.fit_transform(x['Card_Category'])

### one hot encoder :

In [668]:
#x = pd.get_dummies(x)

### Como ficaram os dados?

In [669]:
x.head()

# SEPARAÇÃO TREINO E TESTE

In [670]:
#Separação entre treino e teste
y=df['Attrition_Flag']
#del df 

np.random.seed(50)
x_treino, x_teste, y_treino, y_teste = train_test_split (x, y, test_size = 0.20, random_state = 42)

# FASE DE TREINAMENTO E TESTE

##### OPITEI POR UTILIZAR DOIS MODELOS DE ML QUE SÃO EXTREMAMENTE RAPIDOS, O PRIMIERO É XGB E O SEGUNDO RANDO FOREST
##### TESTEI SVM MAS SE MOSTROU EXTREMAMENTE LENTO, ENTÃO OPTEI POR REMOVELO POIS NÃO HAVERIA TEMPO PARA EXECUÇÃO.
##### DEIXEI O MODELO DESCOBRIR SEUS PARAMETROS PARA NÃO CAUSAR SOBRE AJUSTE TAMBÈM PORQUE NÃO HAVERIA TEMPO PARA TESTES COM DIVERSOS PARAMETROS.

##### VALE COMENTAR QUE UTILIZEI 5 FOLDS e MATRIZ DE CONFUSÃO PARA INTERPRETAR OS RESULTADOS. AO FINAL HÁ UM BREVE COMENTARIO SOBRE OS TESTES E SEUS RESPECTIVOS RESULTADOS.

# TREINAMENTO XGB

In [671]:
from xgboost import XGBClassifier
SVM = XGBClassifier(verbosity = 0)
SVM.fit(x_treino,y_treino)


# TESTE PARA XGB


In [672]:
cv = StratifiedKFold(n_splits = 5, shuffle = True)
y_pred = cross_val_predict(SVM, x_teste, y_teste, cv = cv)
fig, ax = plt.subplots()
sns.heatmap(confusion_matrix(y_teste, y_pred), annot=True, 
            ax=ax, fmt='d', cmap='Reds')
ax.set_title("Matriz de Confusão", fontsize=18)
ax.set_ylabel("True label")
ax.set_xlabel("Predicted Label")
# relatório do modelo
print('Relatório de classificação:\n', classification_report(y_teste, y_pred, digits=4))

# TREINAMENTO RANDOM FOREST

In [673]:
rf = RandomForestClassifier()
rf.fit(x_treino,y_treino)

# TESTE RANDOM FOREST

In [674]:
cv = StratifiedKFold(n_splits = 5, shuffle = True)
y_pred = cross_val_predict(rf, x_teste, y_teste, cv = cv)
fig, ax = plt.subplots()
sns.heatmap(confusion_matrix(y_teste, y_pred), annot=True, 
            ax=ax, fmt='d', cmap='Reds')
ax.set_title("Matriz de Confusão", fontsize=18)
ax.set_ylabel("True label")
ax.set_xlabel("Predicted Label")
# relatório do modelo
print('Relatório de classificação:\n', classification_report(y_teste, y_pred, digits=4))

# CONSIDERAÇÕES FINAIS

## INFELIZMENTE NÃO TIVE TEMPO SUFICIENTE PARA TESTAR OUTROS MODELOS NEM APLICAR ENGENHARIA DE FEATURES QUE TROUXESSEM BENEFICIOS

### XGB SE MOSTROU MAIS EFICAZ OBTENDO 95.46% DE ACCURACY E 95.36% DE F1-SCORE